In [1]:
import os
# Find the latest version of spark 3.2  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.2.2'
spark_version = 'spark-3.2.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease [18.1 kB]
Hit:3 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:4 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:5 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:6 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease [1,581 B]
Hit:8 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Hit:9 http://archive.ubuntu.com/ubuntu focal InRelease
Get:10 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main Sources [2,398 kB]
Get:12 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal/main amd64 Packages [1,136 kB]
Get:13 http://archive.ubuntu.com/ubuntu focal-backports InRe

In [2]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2023-03-18 16:09:29--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  1.67MB/s    in 0.5s    

2023-03-18 16:09:30 (1.67 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [4]:
# Read in data from S3 Buckets
from pyspark import SparkFiles
# url="https://<bucket name>.s3.amazonaws.com/user_data.csv" 
url = "https://meteorite-ml-project.s3.amazonaws.com/landcover.csv"
spark.sparkContext.addFile(url)
landcover_df = spark.read.csv(SparkFiles.get("landcover.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
landcover_df.show()

+------------+-----------------+--------------------+
|state_abbrev|         variable|               value|
+------------+-----------------+--------------------+
|          AL|           FOREST|   64.87214818449999|
|          AL|             GRSL|        19.904489577|
|          AL|             WETL| 0.17698839989999998|
|          AL|           SHRUBL|        1.2225725537|
|          AL|SPARSE_VEGETATION|        0.0347645507|
|          AL|            CROPL|       10.1772646745|
|          AL|            URBAN|        1.6597195122|
|          AL|             BARE|         0.189766099|
|          AL|            WATER|        1.7622864484|
|          CO|           FOREST|       32.9304390513|
|          CO|             GRSL|       31.7712187156|
|          CO|             WETL|0.036524399900000004|
|          CO|           SHRUBL|       17.2468731707|
|          CO|SPARSE_VEGETATION|0.035923292999999995|
|          CO|            CROPL|       15.5319438223|
|          CO|            UR

In [5]:
# url="https://<bucket name>.s3.amazonaws.com/user_payment.csv"
url = 'https://meteorite-ml-project.s3.amazonaws.com/meteorite_main.csv'
spark.sparkContext.addFile(url)
meteoritemain_df = spark.read.csv(SparkFiles.get("meteorite_main.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
meteoritemain_df.show()

+-----+---------------+---------------+----------+-----+----+-----------------+------------------+--------------------+--------------------+---------+-------------+------------+
|   id|           name|       recclass|mass_grams| fall|year|           reclat|           reclong|         geolocation|            geometry|elevation|      country|state_abbrev|
+-----+---------------+---------------+----------+-----+----+-----------------+------------------+--------------------+--------------------+---------+-------------+------------+
| 2276|        Allegan|             H5|   32000.0| Fell|1899|         42.53333|         -85.88333|(42.53333, -85.88...|POINT (-85.88333 ...|    225.0|United States|          MI|
| 5401|        Coleman|             L6|     469.0| Fell|1994|         43.76111|-84.50778000000003|(43.76111, -84.50...|POINT (-84.50778 ...|    218.0|United States|          MI|
|22766|      Rose City|             H5|   10600.0| Fell|1921|         44.51667|            -83.95|  (44.51667,

In [6]:
# url="https://<bucket name>.s3.amazonaws.com/user_payment.csv"
url = 'https://meteorite-ml-project.s3.amazonaws.com/meteorite_types.csv'
spark.sparkContext.addFile(url)
meteoritetypes_df = spark.read.csv(SparkFiles.get("meteorite_types.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
meteoritetypes_df.show()

+--------------------+------------------------+---------------+--------------+
|            recclass|meteorite_class_subclass|meteorite_class|meteorite_type|
+--------------------+------------------------+---------------+--------------+
|                  H5|    Chrondrite - ordi...|     Chrondrite|    Chrondrite|
|                  L6|    Chrondrite - ordi...|     Chrondrite|    Chrondrite|
|                  L5|    Chrondrite - ordi...|     Chrondrite|    Chrondrite|
|     Iron, ungrouped|            Iron - other|           Iron|          Iron|
|           Iron, IVA|         Iron - magmatic|           Iron|          Iron|
|         Iron, IIIAB|         Iron - magmatic|           Iron|          Iron|
|       Eucrite-pmict|             Achrondrite|    Achrondrite|    Chrondrite|
|                  OC|    Chrondrite - ordi...|     Chrondrite|    Chrondrite|
|                  H6|    Chrondrite - ordi...|     Chrondrite|    Chrondrite|
|                  L3|    Chrondrite - ordi...|     

In [7]:
# url="https://<bucket name>.s3.amazonaws.com/user_payment.csv"
url = 'https://meteorite-ml-project.s3.amazonaws.com/state.csv'
spark.sparkContext.addFile(url)
state_df = spark.read.csv(SparkFiles.get("state.csv"), sep=",", header=True, inferSchema=True)

# Show DataFrame
state_df.show()

+------------+--------------------+----+---------+-------------+
|state_abbrev|               state|FIPS|area_sqkm|      country|
+------------+--------------------+----+---------+-------------+
|          AL|             Alabama|   1|   135767|United States|
|          AK|              Alaska|   2|  1723337|United States|
|          AZ|             Arizona|   4|   295234|United States|
|          AR|            Arkansas|   5|   137732|United States|
|          CA|          California|   6|   423967|United States|
|          CO|            Colorado|   8|   269601|United States|
|          CT|         Connecticut|   9|    14357|United States|
|          DE|            Delaware|  10|     6446|United States|
|          DC|District of Columbia|  11|      177|United States|
|          FL|             Florida|  12|   170312|United States|
|          GA|             Georgia|  13|   153910|United States|
|          HI|              Hawaii|  15|    28313|United States|
|          ID|           

Postgres Setup

In [8]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://meteorite-ml-db.cl3jquwwq7kn.us-east-1.rds.amazonaws.com:5432/meteorite_ml_db"
config = {"user":"postgres", 
          "password": "XXXXXXXXXX", 
          "driver":"org.postgresql.Driver"}


In [9]:
# Write DataFrame to landcover table in RDS

landcover_df.write.jdbc(url=jdbc_url, table='landcover', mode=mode, properties=config)

In [10]:
# Write dataframe to meteorite_main table in RDS

meteoritemain_df.write.jdbc(url=jdbc_url, table='meteorite_main', mode=mode, properties=config)

In [11]:
# Write dataframe to meteorite_type table in RDS

meteoritetypes_df.write.jdbc(url=jdbc_url, table='meteorite_type', mode=mode, properties=config)

In [12]:
# Write dataframe to state table in RDS

state_df.write.jdbc(url=jdbc_url, table='state', mode=mode, properties=config)